In [171]:
import xml.etree.ElementTree as ET
test_file = '/home/alex/Downloads/ICSI_plus_NXT/ICSIplus/Contributions/Summarization/extractive/Buw001.extsumm.xml'

In [172]:
def xmlData(file):
    root = ET.parse(file).getroot()
    data = []
    for extsumm in root.iter():
        href = extsumm.get('href')
        if href != None:
            data.append(href.split('#'))
    
    #Strip contents from id(contents)
    for i in range(len(data)):
        data[i][1] = data[i][1].replace('id(' , '')
        data[i][1] = data[i][1].replace(')' , '')
    
    #Data is not always stored cleanly in xml file, this function cleansup odd shaped data
    #that can come in the form of {file: [dial1, dial 2]} -> {file: [dial1, file:dial2]}
        if data[i][1].find('..') != -1:
            item_1, item_2 = data[i][1].split('..')
            #print(item_1)
            #print(item_2)
            one_act = item_1.partition('act')[2]
            two_act = item_2.partition('act')[2]
            data[i][1] = item_1
            for x in range(int(one_act)+1, int(two_act)):
                #print(item_1.partition('act')[0] 
                #            + item_1.partition('act')[1] + str(x))
                data.append([data[i][0],item_1.partition('act')[0] 
                            + item_1.partition('act')[1] + str(x)])
                #print(data[i])
    #for i in data:
    #    print(i)

    return data

raw_data = xmlData(test_file)


In [173]:
def dialogueSearch(files): #Change name
    #General Location for all files in the "dialogue act folder"
    location = '/home/alex/Downloads/ICSI_plus_NXT/ICSIplus/DialogueActs/'
    file_name, dialogue_name = zip(*files) 
    d = dict(zip(range(len(file_name)), file_name))
    all_data = {}
    for key, elem in d.items():
        d[key] ={elem : dialogue_name[key]}
        #print(dialogue_name[key])
        #Now to parse through file and get item information and set up the hierarchy
        # Number: -> File Name -> dialogue act -> Start Time, Stop Time, Participant
        root = ET.parse(location + elem).getroot()
        for dial in root.iter('dialogueact'):
            start_time = dial.get('starttime')
            end_time = dial.get('endtime')
            speaker_id = dial.get('participant')
            dialogue_id = dial.get('{http://nite.sourceforge.net/}id')
            #print(dialogue_id)
        #Some values do not have an associated start
            if dialogue_id == dialogue_name[key]:
                d[key][elem] = {dialogue_id :{'start time' : start_time, 'stop time' : end_time, 
                                                        'Participant' : speaker_id,}}
        #save all values into an all data dictiionary
            all_data.update({dialogue_id :{'start time' : start_time, 'stop time' : end_time, 
                                                        'Participant' : speaker_id,}})
    #for key in all_data.items():
    #    print(key)
    return d
data_hit = dialogueSearch(raw_data)

In [176]:
def match(files):
    #gather the file_name from the dictionary
    for keys in files[0]:
        keys = keys.split('.')
        f_name = keys[0]
    location = '/home/alex/Downloads/ICSI_original_transcripts/transcripts/' + f_name +'.mrt'
    #print(location)

    root = ET.parse(location).getroot()
    hit = []
    #An automated way of moving through dictionaries is needed
    for key, elem in files.items():
        for key, item in elem.items():
            if type(item) is dict:
                d_key = str(list(item)).strip("['']")
                #at this point we have successfully parsed through the dictionary, and are able to
                #see the start time, stop time and participant
                #pluck out participant and start time and see if a match in the mrt file
                #print(d_key)
                d_name = item[d_key]['Participant']
                ds_time = item[d_key]['start time']
                de_time = item[d_key]['stop time']
                #print("---")
                #print(d_key)
                for info in root.iter('Segment'):
                #pluck out participant, start time and stop time
                #open file and search along the lines if par and start and stop time
                    name = info.get('Participant')
                    s_time = info.get('StartTime')
                    e_time = info.get('EndTime')
                    if d_name == name:
                        #print(s_time)
                        if (abs(float(ds_time) - float(s_time)) < 1 
                        and abs(float(de_time) - float(e_time)) < 1):
                            #print(abs(float(ds_time) - float(s_time)))
                            text = info.text.replace("\n","")
                            print(text)
                            hit.append([text, 1])
                            
                    else:
                        text = info.text.replace("\n","")
                        hit.append([text, 0])
    
    
    for i in range(100):
        print(hit[i])

    
match(data_hit)

      Alright. So what we had 
      and other tasks during data collection,    
      Right. I mean, we - because you'd have several people with these pads, you could collect different things.    
      Right. So the- there will be jargon that we he- There'll be transcription errors.    
      But I bet it's a good 
      in most meetings that 
      Whereas if you just 
      I think for this data capture, it would be nice to have a digital camera    
      for a meeting like this, at 
      I think it'd be useful to have a small amount of it just as a proof of concept. 
      I think, though, the 
      O_K. So CrossPads, we're just gonna try it and see what happens.    
      Yeah. What I was gonna 
      And then 
      you're probably not gonna get 
      the question I had about queries was, um, so what we're planning to do is have people look at the summaries and then generate queries?    
      so, the question 
      I'm also wondering if we could ask the - the people a - a q